In [1]:
from pathlib import Path
import sys

import ee
import geemap
from geojson import Polygon

sys.path.append(str(Path.cwd().parent.parent))

from eobathymetry import bathymetry

In [2]:
Map = geemap.Map(center=(52.97, 4.74), zoom=14)

In [3]:
bounds = ee.Geometry(Polygon([[
    (4.574518432617185, 52.91662291147939),
    (4.90548156738281, 52.91662291147939),
    (4.90548156738281, 53.02331125248889),
    (4.574518432617185, 53.02331125248889),
    (4.574518432617185, 52.91662291147939)
]]))
scale = Map.get_scale()
start_date = "2020-01-01"
stop_date = "2021-01-01"

sdb = bathymetry.Bathymetry()

In [4]:
image = sdb.compute_inverse_depth(
    bounds=bounds,
    start=start_date,
    stop=stop_date,
    scale=scale*5,
    missions=["L8", "S2"],
    filter_masked=True,
#     skip_scene_boundary_fix=True
)